# Demonstration avancee Qwen3-TTS 1.7B
## Conception de voix, controle par instructions et clonage vocal

Ce notebook presente les fonctionnalites avancees des modeles Qwen3-TTS 1.7B.

**GPU requis** : Execution > Modifier le type d'execution > GPU (T4 ou superieur)

## 1 - Installation et configuration

In [ ]:
!pip install -U qwen-tts soundfile -q
!pip install flash-attn --no-build-isolation -q

In [ ]:
import torch
import soundfile as sf
import os
from IPython.display import Audio, display, Markdown
print(f"PyTorch : {torch.__version__}")
print(f"CUDA disponible : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU : {torch.cuda.get_device_name(0)}")
os.makedirs("audio_outputs", exist_ok=True)
try:
    import flash_attn
    ATTN_IMPL = "flash_attention_2"
    print("Flash Attention 2 disponible")
except ImportError:
    ATTN_IMPL = "eager"
    print("Flash Attention non disponible")

In [ ]:
def play_audio(wav, sr, filename=None, title=None):
    if filename:
        sf.write(f"audio_outputs/{filename}.wav", wav, sr)
    if title:
        display(Markdown(f"**{title}**"))
    display(Audio(wav, rate=sr))

## PARTIE 1 : Conception de voix (VoiceDesign)

In [ ]:
from qwen_tts import Qwen3TTSModel
torch.cuda.empty_cache()
print("Chargement du modele VoiceDesign...")
voice_design_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modele VoiceDesign charge !")

In [ ]:
print("Generation : Voix de fille anime mignonne")
wavs, sr = voice_design_model.generate_voice_design(text="Coucou tout le monde ! Je suis tellement contente de vous rencontrer aujourd'hui ! On va bien s'amuser ensemble !", language="French", instruct="Voix feminine jeune, environ 16 ans, tres mignonne et energique. Aigue avec un ton vif et joyeux. Parle avec enthousiasme et excitation, comme un personnage d'anime.")
play_audio(wavs[0], sr, "vd_01_anime_girl_fr", "Voix de fille anime mignonne")

In [ ]:
print("Generation : Voix de narrateur de documentaire")
wavs, sr = voice_design_model.generate_voice_design(text="Dans les profondeurs de l'ocean, la ou la lumiere du soleil ne penetre jamais, se cache un monde peuple de creatures extraordinaires.", language="French", instruct="Voix masculine, 50-60 ans, baryton profond. Parle lentement et deliberement avec gravite. Style narrateur de documentaire professionnel. Calme, autoritaire et captivant.")
play_audio(wavs[0], sr, "vd_02_narrator_fr", "Narrateur de documentaire")

In [ ]:
print("Generation : Voix d'adolescent nerveux")
wavs, sr = voice_design_model.generate_voice_design(text="Euh, s-salut... Je me demandais si peut-etre... tu vois... si t'es pas trop occupee... on pourrait reviser ensemble ?", language="French", instruct="Homme, 17 ans, registre tenor. Nerveux et timide, voix legerement tremblante. Parle avec hesitation et des pauses, trebuchant sur les mots.")
play_audio(wavs[0], sr, "vd_03_nervous_teen_fr", "Adolescent nerveux")

In [ ]:
print("Generation : Voix de mechant mysterieux")
wavs, sr = voice_design_model.generate_voice_design(text="Tu pensais pouvoir t'echapper ? Comme c'est delicieusement naif. Le jeu ne fait que commencer.", language="French", instruct="Voix masculine, age moyen, grave et soyeuse. Parle lentement avec des sous-entendus menacants. Confiant et legerement moqueur.")
play_audio(wavs[0], sr, "vd_04_villain_fr", "Mechant mysterieux")

In [ ]:
test_line = "Je n'arrive pas a croire que ca se passe vraiment en ce moment."
voice_designs = [("Enfant excite", "Fille de 8 ans, voix tres aigue et excitee. Parle vite avec enthousiasme."), ("Adulte epuise", "Femme de 35 ans, fatiguee et exasperee. Soupire, parle lentement."), ("Ancien choque", "Homme de 70 ans, voix rauque, choque et incredule."), ("Ado sarcastique", "Fille de 16 ans, sarcasme qui degouline. Monotone.")]
print(f"Texte : {test_line}")
for i, (name, instruct) in enumerate(voice_designs):
    print(f"{name}")
    wavs, sr = voice_design_model.generate_voice_design(text=test_line, language="French", instruct=instruct)
    play_audio(wavs[0], sr, f"vd_05_same_text_fr_{i+1}")

## PARTIE 2 : Controle par instructions (CustomVoice)

In [ ]:
torch.cuda.empty_cache()
print("Chargement du modele CustomVoice 1.7B...")
custom_voice_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modele CustomVoice 1.7B charge !")
print(f"Locuteurs disponibles : {', '.join(custom_voice_model.get_supported_speakers())}")

In [ ]:
emotion_text = "Je viens d'apprendre ce qui s'est passe hier."
speaker = "Ryan"
emotions = [("Joyeux", "Ton tres joyeux et excite"), ("Triste", "Triste et melancolique, voix legerement brisee"), ("En colere", "En colere et frustre, parlant avec force"), ("Apeure", "Effraye et anxieux, voix tremblante"), ("Neutre", "")]
print(f"Texte : {emotion_text}")
for emotion_name, instruct in emotions:
    print(f"{emotion_name}")
    wavs, sr = custom_voice_model.generate_custom_voice(text=emotion_text, language="French", speaker=speaker, instruct=instruct)
    play_audio(wavs[0], sr, f"cv_01_emotion_fr_{emotion_name.lower().replace(' ', '_')}")

In [ ]:
style_text = "S'il vous plait, faites silence, le bebe dort."
speaker = "Serena"
styles = [("Chuchotement", "Chuchotant tres doucement et silencieusement"), ("Fort", "Parlant fort et clairement, projetant la voix"), ("Rapide", "Parlant tres vite, rythme presse"), ("Lent", "Parlant tres lentement et deliberement"), ("Dramatique", "Tres dramatique et theatrical")]
print(f"Texte : {style_text}")
for style_name, instruct in styles:
    print(f"{style_name}")
    wavs, sr = custom_voice_model.generate_custom_voice(text=style_text, language="French", speaker=speaker, instruct=instruct)
    play_audio(wavs[0], sr, f"cv_02_style_fr_{style_name.lower()}")

In [ ]:
roleplay_scenarios = [("Ryan", "Bienvenue a ce match de championnat ! La tension ici est absolument electrique !", "Commentateur sportif, tres energique et excite", "Commentateur"), ("Serena", "Notre chiffre d'affaires trimestriel a augmente de quinze pour cent.", "Presentation professionnelle, confiante et claire", "Presentatrice"), ("Aiden", "Et alors le dragon s'est retourne vers notre groupe... lancez les des !", "Maitre du donjon, mysterieux et dramatique", "Donjon"), ("Ono_Anna", "La recette du jour, c'est un delicieux omelette au riz !", "Animatrice cuisine joyeuse et chaleureuse", "Cuisine")]
print("SCENARIOS DE JEU DE ROLE")
for speaker, text, instruct, title in roleplay_scenarios:
    print(f"{title} - {speaker}")
    wavs, sr = custom_voice_model.generate_custom_voice(text=text, language="French", speaker=speaker, instruct=instruct)
    play_audio(wavs[0], sr, f"cv_03_roleplay_fr_{title.lower()}")

## PARTIE 3 : Clonage vocal (Base)

In [ ]:
torch.cuda.empty_cache()
print("Chargement du modele Base (clonage vocal)...")
clone_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-Base", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modele Base charge !")

In [ ]:
# IMPORTANT : Remplacez par votre propre fichier audio
ref_audio_url = "/content/votre_audio_reference.mp3"
ref_text = "Ceci est la transcription exacte de ce qui est dit dans votre audio de reference."
print(f"Audio de reference : {ref_audio_url}")
new_texts = ["Bonjour, ceci est un test de clonage vocal en francais.", "Le temps aujourd'hui est magnifique, il fait beau et chaud.", "Cette technologie de clonage vocal est vraiment impressionnante."]
for i, text in enumerate(new_texts):
    print(f"Generation : {text}")
    wavs, sr = clone_model.generate_voice_clone(text=text, language="French", ref_audio=ref_audio_url, ref_text=ref_text)
    play_audio(wavs[0], sr, f"clone_01_sample_fr_{i+1}")

## PARTIE 4 : Conception puis Clonage

In [ ]:
torch.cuda.empty_cache()
print("Chargement du modele VoiceDesign...")
design_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modele VoiceDesign charge !")

In [ ]:
print("ETAPE 1 : Concevoir la voix du personnage")
character_instruct = "Homme, age environ 70 ans, voix grave et rocailleuse avec sagesse. Parle lentement et pensivement. Chaleureux mais autoritaire."
reference_text = "Ah, jeune apprenti, tu cherches la connaissance des arts anciens. Tres bien, je vais t'enseigner les secrets de la magie."
ref_wavs, sr = design_model.generate_voice_design(text=reference_text, language="French", instruct=character_instruct)
sf.write("audio_outputs/sorcier_reference.wav", ref_wavs[0], sr)
print("Audio de reference genere !")
play_audio(ref_wavs[0], sr, title="Voix du vieux sorcier")

In [ ]:
torch.cuda.empty_cache()
print("ETAPE 2 : Creer le clone vocal du sorcier")
clone_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-Base", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
sorcier_prompt = clone_model.create_voice_clone_prompt(ref_audio=(ref_wavs[0], sr), ref_text=reference_text)
print("Prompt vocal du sorcier cree avec succes !")

In [ ]:
print("ETAPE 3 : Generer les dialogues du sorcier")
dialogues_sorcier = ["Le chemin de la magie est long et perilleux, mon enfant.", "En sept cents ans de vie, j'ai appris que la patience est la plus grande des vertus.", "Mefie-toi de la foret sombre qui se trouve a l'est du royaume.", "Tu as bien travaille aujourd'hui, jeune apprenti. Je suis fier de toi.", "Maintenant, commencons ton entrainement aux arts mystiques."]
for i, replique in enumerate(dialogues_sorcier):
    print(f"Replique {i+1} : {replique}")
    wavs, sr = clone_model.generate_voice_clone(text=replique, language="French", voice_clone_prompt=sorcier_prompt)
    play_audio(wavs[0], sr, f"sorcier_replique_fr_{i+1}")

## Telechargement des fichiers

In [ ]:
import shutil
from google.colab import files
shutil.make_archive("qwen3_tts_outputs_fr", "zip", "audio_outputs")
print("Fichiers compresses !")
for f in sorted(os.listdir("audio_outputs")):
    if f.endswith(".wav"):
        print(f"  {f}")
files.download("qwen3_tts_outputs_fr.zip")